In [1]:
import pandas as pd
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import when
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import ensemble
import importlib
importlib.reload(ensemble)
from ensemble import *

In [2]:
#.config("spark.executor.memoryOverhead", "64g")\
spark = SparkSession\
    .builder\
    .appName("ensemble")\
    .config("spark.executor.heartbeatInterval", "60s")\
    .config("spark.executor.memory", "32g")\
    .config("spark.driver.memory", "32g")\
    .config("spark.driver.maxResultSize", "64g")\
    .config("spark.sql.crossJoin.enabled", True)\
    .getOrCreate()
sc = spark.sparkContext

In [3]:
base_dir = "/tmp/supersecret_ensemble/"

train_pred_files = {
    "like": {
        "mf": base_dir + "mf/train_prediction_like.tsv", 
        "rf": base_dir + "rf/rf_like_out_ensembletrain_format.csv", 
        "nncf": base_dir + "nncf/like.supersecret_ensembletrain5k_bootstrap.csv"
    },
    "reply": {
        "mf": base_dir + "mf/train_prediction_reply.tsv", 
        "rf": base_dir + "rf/rf_reply_out_ensembletrain_format.csv", 
        "nncf": base_dir + "nncf/reply.supersecret_ensembletrain5k_bootstrap.csv"
    },
    "retweet": {
        "mf": base_dir + "mf/train_prediction_retweet.tsv", 
        "rf": base_dir + "rf/rf_retweet_out_ensembletrain_format.csv", 
        "nncf": base_dir + "nncf/retweet.supersecret_ensembletrain5k_bootstrap.csv"
    },
    "retweet_with_comment": {
        "mf": base_dir + "mf/train_prediction_retweet_with_comment.tsv", 
        "rf": base_dir + "rf/rf_retweet_comment_out_ensembletrain_format.csv", 
        "nncf": base_dir + "nncf/retweet_comment.supersecret_ensembletrain5k_bootstrap.csv"
    },
}

test_pred_files = {
    "like": {
        "mf": base_dir + "mf/test_prediction_like.tsv", 
        "rf": base_dir + "rf/rf_like_out_test_format.csv", 
        "nncf": base_dir + "nncf/like.supersecret_test5k_bootstrap.csv"
    },
    "reply": {
        "mf": base_dir + "mf/test_prediction_reply.tsv", 
        "rf": base_dir + "rf/rf_reply_out_test_format.csv", 
        "nncf": base_dir + "nncf/reply.supersecret_test5k_bootstrap.csv"
    },
    "retweet": {
        "mf": base_dir + "mf/test_prediction_retweet.tsv", 
        "rf": base_dir + "rf/rf_retweet_out_test_format.csv", 
        "nncf": base_dir + "nncf/retweet.supersecret_test5k_bootstrap.csv"
    },
    "retweet_with_comment": {
        "mf": base_dir + "mf/test_prediction_retweet_with_comment.tsv", 
        "rf": base_dir + "rf/rf_retweet_comment_out_test_format.csv", 
        "nncf": base_dir + "nncf/retweet_comment.supersecret_test5k_bootstrap.csv"
    },
}
    
train_label_file = base_dir + "labels/train_labels.tsv"
test_label_file = base_dir + "labels/test_labels.tsv"
schemas = {
    "nncf": StructType([
    StructField("engaging_user_id", StringType()),
    StructField("tweet_id", StringType()),
    StructField("prediction", DoubleType())
    ]),
    "rf": StructType([
        StructField("tweet_id", StringType()),
        StructField("engaging_user_id", StringType()),
        StructField("prediction", DoubleType())
    ])
}

ens = ensemble(spark, sc)

In [4]:
def file_exist(path):
    fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(sc._jsc.hadoopConfiguration())
    return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))
for engagement in train_pred_files:
    for model_name in train_pred_files[engagement]:
        if not file_exist(train_pred_files[engagement][model_name]):
            print("File does not exist for engagement {} and model_name {}".format(engagement, model_name))
        if not file_exist(test_pred_files[engagement][model_name]):
            print("File does not exist for engagement {} and model_name {}".format(engagement, model_name))

In [5]:
coef_list = []
evals = {}
for engagement in train_pred_files:
    #lr_model = LinearRegressionModel.load(base_dir + "model_" + engagement)
    lr_model = ens.train(train_pred_files[engagement], schemas, train_label_file, engagement)
    lr_model.save(base_dir + "new_model_" + engagement)
    coef_list.append(list(lr_model.coefficients) + [lr_model.intercept])

    eval_df = ens.test_evaluate(lr_model, test_pred_files[engagement], schemas, test_label_file, engagement, thresholds=[0.1, 0.3, 0.5, 0.7, 0.9])
    evals[engagement] = eval_df
    eval_df.round(3).to_csv("ensemble_{}.csv".format(engagement), index=False)

In [6]:
coef_list = [[e] + coef for (coef, e) in zip(coef_list, train_pred_files.keys())]
coef_df = pd.DataFrame(coef_list, columns=["engagement", "mf", "rf", "nncf", "intercept"])
coef_df.head()

,engagement,mf,rf,nncf,intercept
0,like,0.340659,0.824783,0.007285,0.052271
1,reply,0.000000,0.245472,-0.011267,0.002892
2,retweet,0.766905,0.891979,0.016441,0.009324
3,retweet_with_comment,0.000000,0.003317,0.000539,0.006377


In [7]:
coef_df.to_csv("coefficients.csv", index=False)

In [8]:
evals['retweet_with_comment']

,areaUnderPR-0.1,areaUnderPR-0.3,areaUnderPR-0.5,areaUnderPR-0.7,areaUnderPR-0.9,log_loss,model_name
0,0.007594,0.007594,0.007594,0.007594,0.007594,NaN,mf
1,0.004598,0.009196,0.009196,0.009196,0.009196,0.052119,rf
2,0.010245,0.010245,0.010245,0.010245,0.010245,3.861197,nncf
3,0.007594,0.007594,0.007594,0.007594,0.007594,0.044610,Ensemble


In [9]:
evals['reply']

,areaUnderPR-0.1,areaUnderPR-0.3,areaUnderPR-0.5,areaUnderPR-0.7,areaUnderPR-0.9,log_loss,model_name
0,0.026703,0.026703,0.026703,0.026703,0.026703,NaN,mf
1,0.015094,0.030188,0.030188,0.030188,0.030188,0.131628,rf
2,0.029350,0.029360,0.029360,0.029360,0.029360,2.317991,nncf
3,0.066025,0.026703,0.026703,0.026703,0.026703,0.122242,Ensemble


In [10]:
evals['retweet']

,areaUnderPR-0.1,areaUnderPR-0.3,areaUnderPR-0.5,areaUnderPR-0.7,areaUnderPR-0.9,log_loss,model_name
0,0.089417,0.089417,0.089417,0.089417,0.089417,NaN,mf
1,0.148980,0.317230,0.411653,0.107957,0.107957,0.322430,rf
2,0.108827,0.108827,0.108827,0.108827,0.108827,4.024650,nncf
3,0.089417,0.089417,0.131815,0.254896,0.332955,0.670192,Ensemble


In [11]:
evals['retweet_with_comment']

,areaUnderPR-0.1,areaUnderPR-0.3,areaUnderPR-0.5,areaUnderPR-0.7,areaUnderPR-0.9,log_loss,model_name
0,0.007594,0.007594,0.007594,0.007594,0.007594,NaN,mf
1,0.004598,0.009196,0.009196,0.009196,0.009196,0.052119,rf
2,0.010245,0.010245,0.010245,0.010245,0.010245,3.861197,nncf
3,0.007594,0.007594,0.007594,0.007594,0.007594,0.044610,Ensemble
